# Python Basics


In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf


DJ = pd.read_csv("csv/20M/XAUUSD_M20_202202140100_202204282340.csv",delim_whitespace=True)
timeFrame = int(input('TimeFrame:'))
CandlesInDay = 24 * (60//timeFrame)
DJ.insert(7, '<CANDLESIZE>',value=0)
DJ.drop(['<HIGH>','<LOW>','<TICKVOL>','<VOL>','<SPREAD>'],axis=1,inplace=True)
DJ['<CANDLESIZE>'] =  DJ['<CLOSE>'] > DJ['<OPEN>']






In [11]:
def candleToTime(j):
    k=j//minuteMult
    sth=timeFrame*(j%minuteMult)
    return '{:02d}:{:02d}:00'.format(k,sth)

uniqueDays = DJ.drop_duplicates(subset='<DATE>')
uniqueDays = pd.DataFrame(uniqueDays)

uniqueDaysCount=uniqueDays.shape[0]


newDF = pd.DataFrame(columns=['<DATE>','<TIME>','<OPEN>','<CLOSE>','<CANDLESIZE>'])

minuteMult = CandlesInDay//24



for i in range(uniqueDays.shape[0]):
    for j in range(CandlesInDay):
        
        newDF.loc[(i*CandlesInDay)+j]=[uniqueDays.iloc[i][0],candleToTime(j),0,0,0]


In [12]:
newestDF = newDF.merge(DJ,on=['<DATE>','<TIME>'],how='left')

newestDF.drop(columns=['<OPEN>_x','<CLOSE>_x','<CANDLESIZE>_x','<CLOSE>_y','<OPEN>_y'],inplace=True)
newestDF['<CANDLESIZE>_y'] = newestDF['<CANDLESIZE>_y'].fillna(0)



In [13]:
list = []
for i in range(len(newestDF)):
    date = newestDF.iloc[i][0]
    time = newestDF.iloc[i][1]
    GreenOrRed = newestDF.iloc[i][2]
    list.append([date, time, GreenOrRed])


In [14]:
days = []
for i in range(uniqueDaysCount): 
    days.append(list[i*CandlesInDay: CandlesInDay*(i+1)])


In [15]:


def eachDayMat(day):
    ansMatrix = np.zeros((CandlesInDay,CandlesInDay),dtype=int)
    for i in range(len(day)) : 
        for j in range (i,len(day)):
            ansMatrix[i][j]=not(day[i][2] ^ day[j][2])

    return ansMatrix

ansVector = []
for i in range(len(days)):
    ansVector.append(eachDayMat(days[i]))

final = np.sum(ansVector, axis=0)
# final-=len(days)//2

In [21]:
bias=.63
# float(input('Bias:')) 



for i in range(len(final)):
    for j in range (i,len(final)):
        if (final[i][j]>(bias*uniqueDaysCount) or final[i][j]<((1-bias)*uniqueDaysCount) )and (not i==j):
            firstouterCandles = 60//timeFrame
            lastouterCandles = CandlesInDay - firstouterCandles
            if(i<firstouterCandles or j<firstouterCandles or i>lastouterCandles or j>lastouterCandles):
                continue
            else:
                if(j-i==1):
                    
                    print(f"Consecutive: {i}-{j}",f"{candleToTime(i)}-{candleToTime(j)}",f"{int(final[i][j]/uniqueDaysCount*10000)/100}%")
                    
                





Consecutive: 7-8 02:20:00-02:40:00 67.92%
Consecutive: 22-23 07:20:00-07:40:00 35.84%
Consecutive: 43-44 14:20:00-14:40:00 35.84%
Consecutive: 68-69 22:40:00-23:00:00 30.18%
